In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

Se tiene un RDD con información de vuelos programados con la forma (número de vuelo, código de aerolínea, código de aeropuerto de salida, código de aeropuerto de llegada, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM). A su vez, se cuenta con el registro actualizado del estado de los vuelos que fueron ocurriendo, con la forma (número de vuelo, aerolínea, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM, estado). En base al estado, podría contar con algún dato en blanco, por ejemplo si el vuelo fue cancelado no tendrá información de fechas y horas, si el vuelo se encuentra aún en curso, no contendrá información de la llegada.

In [15]:
vuelos_programados_data = [
    (5,1,1,10,"20200503","20:00","20200504","03:30"),
    (6,1,3,11,"20200503","21:00","20200504","16:30"),
    (5,2,1,11,"20201203","13:12","20201203","15:15"),
    (5,2,1,12,"20200504","07:40","20200504","20:00"),
    (7,1,1,13,"20190405","08:55","20190406","19:30")
]

vuelos_ocurridos_data = [
    (1,1,1,10,"20190503","20:00","20190504","03:30"),
    (2,1,1,11,"20191012","20:00","20191012","23:59"),
    (1,2,1,11,"20180503","19:00","20180504","01:25"),
    (2,2,1,13,"20170503","20:00","20170504","03:30")
]

vuelos_programados = sc.parallelize(vuelos_programados_data)

vuelos_programados = vuelos_programados.toDF(['nvuelo',"codigo","codigo_aeropuerto","cod_aerop_llegada","fecha_salida",\
 "hora_salida", "fecha_llegada", "hora_llegada"]).withColumn("fecha_salida", (col('fecha_salida').cast("date")))\
.withColumn("fecha_llegada", (col('fecha_llegada').cast("date"))).withColumn("hora_salida", (col("hora_salida")\
.cast("timeStamp"))).withColumn("hora_llegada", (col("hora_llegada").cast("timeStamp"))).rdd

vuelos_ocurridos = sc.parallelize(vuelos_ocurridos_data)

vuelos_ocurridos = vuelos_ocurridos.toDF(['nvuelo',"codigo","codigo_aeropuerto","cod_aerop_llegada","fecha_salida",\
 "hora_salida", "fecha_llegada", "hora_llegada"]).withColumn("fecha_salida", (col('fecha_salida').cast("date")))\
.withColumn("fecha_llegada", (col('fecha_llegada').cast("date"))).withColumn("hora_salida", (col("hora_salida")\
.cast("timeStamp"))).withColumn("hora_llegada", (col("hora_llegada").cast("timeStamp"))).rdd

In [16]:
vuelos_programados.collect()

[Row(nvuelo=5, codigo=1, codigo_aeropuerto=1, cod_aerop_llegada=10, fecha_salida=None, hora_salida=datetime.datetime(2020, 10, 25, 20, 0), fecha_llegada=None, hora_llegada=datetime.datetime(2020, 10, 25, 3, 30)),
 Row(nvuelo=6, codigo=1, codigo_aeropuerto=3, cod_aerop_llegada=11, fecha_salida=None, hora_salida=datetime.datetime(2020, 10, 25, 21, 0), fecha_llegada=None, hora_llegada=datetime.datetime(2020, 10, 25, 16, 30)),
 Row(nvuelo=5, codigo=2, codigo_aeropuerto=1, cod_aerop_llegada=11, fecha_salida=None, hora_salida=datetime.datetime(2020, 10, 25, 13, 12), fecha_llegada=None, hora_llegada=datetime.datetime(2020, 10, 25, 15, 15)),
 Row(nvuelo=5, codigo=2, codigo_aeropuerto=1, cod_aerop_llegada=12, fecha_salida=None, hora_salida=datetime.datetime(2020, 10, 25, 7, 40), fecha_llegada=None, hora_llegada=datetime.datetime(2020, 10, 25, 20, 0)),
 Row(nvuelo=7, codigo=1, codigo_aeropuerto=1, cod_aerop_llegada=13, fecha_salida=None, hora_salida=datetime.datetime(2020, 10, 25, 8, 55), fecha_

In [17]:
vuelos_ocurridos.collect()

[Row(nvuelo=1, codigo=1, codigo_aeropuerto=1, cod_aerop_llegada=10, fecha_salida=None, hora_salida=datetime.datetime(2020, 10, 25, 20, 0), fecha_llegada=None, hora_llegada=datetime.datetime(2020, 10, 25, 3, 30)),
 Row(nvuelo=2, codigo=1, codigo_aeropuerto=1, cod_aerop_llegada=11, fecha_salida=None, hora_salida=datetime.datetime(2020, 10, 25, 20, 0), fecha_llegada=None, hora_llegada=datetime.datetime(2020, 10, 25, 23, 59)),
 Row(nvuelo=1, codigo=2, codigo_aeropuerto=1, cod_aerop_llegada=11, fecha_salida=None, hora_salida=datetime.datetime(2020, 10, 25, 19, 0), fecha_llegada=None, hora_llegada=datetime.datetime(2020, 10, 25, 1, 25)),
 Row(nvuelo=2, codigo=2, codigo_aeropuerto=1, cod_aerop_llegada=13, fecha_salida=None, hora_salida=datetime.datetime(2020, 10, 25, 20, 0), fecha_llegada=None, hora_llegada=datetime.datetime(2020, 10, 25, 3, 30))]